In [ ]:
!pip install russian-tagsets
!pip install sklearn-crfsuite

In [2]:
import re
from copy import deepcopy
import os

import pandas as pd
import numpy as np

import sklearn_crfsuite

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

from russian_tagsets import converters
from pymorphy2 import MorphAnalyzer
import conllu

from tqdm import tqdm

In [3]:
p_analyser = MorphAnalyzer()

In [4]:
t = p_analyser.parse("стекло")[0].tag
t

OpencorporaTag('NOUN,inan,neut sing,nomn')

In [5]:
to_ud = converters.converter('opencorpora-int', 'ud20')
to_ud(str(t))

'NOUN Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing'

In [6]:
def __conllu_word_to_dict(tokens, pos) -> dict:
    res = {}
    for i in range(pos-2, pos+3):
        for feat in ['lemma', 'upos', 'feats']:
            if i >= 0 and i < len(tokens):
                res[feat+str(i-pos)] = tokens[i][feat]
            else:
                res[feat+str(i-pos)] = 'none'
    return res


def conllu_word_to_dict(tokens, pos) -> dict:
    res = {}
    out_pos = tokens[pos]['upos']
    for i in range(pos-2, pos+3):
        if i >= 0 and i < len(tokens):
            m_res = p_analyser.parse(tokens[i]['form'])
            m_pos = 0
            while m_pos < len(m_res) and m_pos < 3:
                res['lemma'+str(i-pos)+str(m_pos)] = m_res[m_pos].normal_form
                feats = to_ud(str(m_res[m_pos].tag))
                upos, feats = feats.split(' ')
                res['upos'+str(i-pos)+str(m_pos)] = upos
                res['feats'+str(i-pos)+str(m_pos)] = feats
                m_pos += 1
                
            while m_pos < 3:
                res['lemma'+str(i-pos)+str(m_pos)] = 'none'
                res['upos'+str(i-pos)+str(m_pos)] = 'none'
                res['feats'+str(i-pos)+str(m_pos)] = 'none'
                m_pos += 1
        else:
            for m_pos in range(3):
                res['lemma'+str(i-pos)+str(m_pos)] = 'none'
                res['upos'+str(i-pos)+str(m_pos)] = 'none'
                res['feats'+str(i-pos)+str(m_pos)] = 'none'
        
    return res, out_pos

In [7]:
ud_path = "/home/edward/projects/Alien_bases/Universal Dependencies/ud-treebanks-v2.8/UD_Russian-SynTagRus/"
in_file = open(ud_path+'ru_syntagrus-ud-train.conllu', 'rt')
train_connlu_iter = conllu.parse_incr(in_file)

In [8]:
i = 0
for sent in train_connlu_iter:
    print(sent[0], sent, i)
    i += 1
    if i > 5:
        break

Анкета TokenList<Анкета, ., metadata={sent_id: "2003Anketa.xml_1", text: "Анкета."}> 0
Начальник TokenList<Начальник, областного, управления, связи, Семен, Еремеевич, был, человек, простой, ,, приходил, на, работу, всегда, вовремя, ,, здоровался, с, секретаршей, за, руку, и, иногда, даже, писал, в, стенгазету, заметки, под, псевдонимом, ", Муха, ", ., metadata={sent_id: "2003Anketa.xml_2", text: "Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха"."}> 1
В TokenList<В, приемной, его, с, утра, ожидали, посетители, ,, -, кое-кто, с, важными, делами, ,, а, кое-кто, и, с, такими, ,, которые, легко, можно, было, решить, в, нижестоящих, инстанциях, ,, не, затрудняя, Семена, Еремеевича, ., metadata={sent_id: "2003Anketa.xml_3", text: "В приемной его с утра ожидали посетители, - кое-кто с важными делами, а кое-кто и с такими, которые легко мо

In [9]:
res_sents = []
res_poses = []

for sent in tqdm(train_connlu_iter):
    tagged_sent = []
    sent_poses = []
    for i in range(len(sent)):
        tags, pos = conllu_word_to_dict(sent, i)
        tagged_sent.append(tags)
        sent_poses.append(pos)
    res_sents.append(tagged_sent)
    res_poses.append(sent_poses)

48808it [10:00, 81.29it/s] 


In [36]:
list(zip(tagged_sent, sent_poses))

[({'lemma-20': 'none',
   'upos-20': 'none',
   'feats-20': 'none',
   'lemma-21': 'none',
   'upos-21': 'none',
   'feats-21': 'none',
   'lemma-22': 'none',
   'upos-22': 'none',
   'feats-22': 'none',
   'lemma-10': 'none',
   'upos-10': 'none',
   'feats-10': 'none',
   'lemma-11': 'none',
   'upos-11': 'none',
   'feats-11': 'none',
   'lemma-12': 'none',
   'upos-12': 'none',
   'feats-12': 'none',
   'lemma00': '-',
   'upos00': 'PUNCT',
   'feats00': '_',
   'lemma01': 'none',
   'upos01': 'none',
   'feats01': 'none',
   'lemma02': 'none',
   'upos02': 'none',
   'feats02': 'none',
   'lemma10': 'да',
   'upos10': 'PART',
   'feats10': '_',
   'lemma11': 'да',
   'upos11': 'CCONJ',
   'feats11': '_',
   'lemma12': 'да',
   'upos12': 'INTJ',
   'feats12': '_',
   'lemma20': ',',
   'upos20': 'PUNCT',
   'feats20': '_',
   'lemma21': 'none',
   'upos21': 'none',
   'feats21': 'none',
   'lemma22': 'none',
   'upos22': 'none',
   'feats22': 'none'},
  'PUNCT'),
 ({'lemma-20': 'no

In [10]:
model = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.1, 
                                   max_iterations=200, all_possible_transitions=True)

In [13]:
%%time
try:
    model.fit(res_sents, res_poses)
except:
    pass

CPU times: user 9min 52s, sys: 627 ms, total: 9min 52s
Wall time: 16min 44s


In [14]:
model.predict([tagged_sent])

[['CCONJ',
  'PRON',
  'VERB',
  'PRON',
  'ADV',
  'CCONJ',
  'PUNCT',
  'VERB',
  'VERB',
  'PUNCT',
  'VERB',
  'ADV',
  'NUM',
  'NOUN',
  'PUNCT']]

In [15]:
sent

TokenList<Но, кто-то, идет, мне, навстречу, и, ,, может, быть, ,, прошел, уже, пол, пути, …, metadata={sent_id: "uppsalaNagibin_4.xml_71", text: "Но кто-то идет мне навстречу и, может быть, прошел уже полпути…"}>